# MLA 703. RNN-LSTM [Analyse de sentiment]

In [ ]:
# Dans ce notebook, nous allons nous intéresser à des tâches d'analyse de sentiments
# -> c'est à dire prédire un label de sentiment (ici positif ou négatif) à partir d'un texte

# Ce notebook vise à approfondir : 
# - L'application du DL sur des données textuelles
# - La compréhension des architectures RNN avancées comme les LSTM et les mécanismes d'attention
#

# 1. Importation des modules

In [21]:
# On importe les librairies usuelless
import math
import numpy as np
import matplotlib.pyplot as plt

# On désactive les warnings
import warnings
warnings.filterwarnings('ignore')


## 1. Charger les données

In [22]:
import pandas as pd
#import os
#os.system("kaggle datasets download -d utathya/imdb-review-dataset")

# On lit le fichier csv de la base d'avis de IMDB (qui doit être préalablement être placé dans le dossier ./data/)
data = pd.read_csv('./imdb_master.csv',encoding="latin-1")


In [23]:
# On visualise les données brutes
print(data)

       Unnamed: 0   type  ...  label         file
0               0   test  ...    neg      0_2.txt
1               1   test  ...    neg  10000_4.txt
2               2   test  ...    neg  10001_1.txt
3               3   test  ...    neg  10002_3.txt
4               4   test  ...    neg  10003_3.txt
...           ...    ...  ...    ...          ...
99995       99995  train  ...  unsup   9998_0.txt
99996       99996  train  ...  unsup   9999_0.txt
99997       99997  train  ...  unsup    999_0.txt
99998       99998  train  ...  unsup     99_0.txt
99999       99999  train  ...  unsup      9_0.txt

[100000 rows x 5 columns]


In [24]:
# On visualise le tableau de données (en utilisant les méthodes de data)
data.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


## 2. Formater/Préparer les données

In [25]:
# FORMATAGE DES DONNEES

# On supprimes les données sans label
data = data[data.label != 'unsup']

# On reformate les labels et entiers (pos = 0; neg = 1)
data['label'] = data['label'].map({'pos': 1, 'neg': 0})

# On supprime les champs inutiles
data = data.drop(['Unnamed: 0','file'],axis=1)

# On visualise le tableau de données
data.head()

,type,review,label
0,test,Once again Mr. Costner has dragged out a movie...,0
1,test,This is an example of why the majority of acti...,0
2,test,"First of all I hate those moronic rappers, who...",0
3,test,Not even the Beatles could write songs everyon...,0
4,test,Brass pictures (movies is not a fitting word f...,0


In [26]:
# 2) FILTRER LES DONNEES
#
# Le nettoyage des données est une étape essentielle de pré-traitement pour optimiser l'apprentissage en aval
#
# Pour le texte, il s'agit de : 
# - normer le texte,  
# - segmenter le texte en mots 
# - supprimer les mots peu porteurs d'information
# - puis détermines les lemmes, de , etc...
#

# On importe la librarie nltk dédiée aux traitement du langage naturel
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

# On peut utiliser l'environnement nltk spécialisé pour le traitement de texte
# (WordNet, stopwords pré-appris sur des bases de données)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

# On pré-traite le texte pour normaliser le texte en entrée
# - On retire certains caractères spéciaux
# - On force la casse en minuscule
# - on retire tous les mots qui sont des "stop_words"
# - on lemmatise les mots (~les formes canoniques)
def preprocess_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

# On pré-traite les données et on les place dans le champs "Processed reviews"
data['Processed_Reviews'] = data.review.apply(lambda x: preprocess_text(x))

# On visualise les données
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,type,review,label,Processed_Reviews
0,test,Once again Mr. Costner has dragged out a movie...,0,mr costner ha drag movie far longer necessary ...
1,test,This is an example of why the majority of acti...,0,example majority action film generic bore real...
2,test,"First of all I hate those moronic rappers, who...",0,first hate moronic rapper couldnt act gun pres...
3,test,Not even the Beatles could write songs everyon...,0,even beatles could write song everyone like al...
4,test,Brass pictures (movies is not a fitting word f...,0,brass picture movie fit word really somewhat b...


In [29]:
# On découpe les phrases en "mots"

from tensorflow.keras.preprocessing.text import Tokenizer

# On ne retient que les 5,000 tokens les plus fréquents (dans le texte)
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['Processed_Reviews'])

# On vectorise les mots

from tensorflow.keras.preprocessing.sequence import pad_sequences

print("La phrase à encoder est : \n  {} \n" .format(data['Processed_Reviews'][0]))

# On vectorise les tokens en séquences numériques
list_tokenized_train = tokenizer.texts_to_sequences(data['Processed_Reviews'])

print("La phrase vectorisée est : \n {} \n". format(list_tokenized_train[0]))

La phrase à encoder est : 
  mr costner ha drag movie far longer necessary aside terrific sea rescue sequence care character u ghost closet costners character realize early forget much later time care character really care cocky overconfident ashton kutcher problem come kid think better anyone else around show sign clutter closet obstacle appear win costner finally well past half way point stinker costner tell u kutchers ghost tell kutcher drive best prior inkling foreshadow magic wa could keep turn hour 

La phrase vectorisée est : 
 [334, 10, 981, 2, 143, 1042, 1556, 1047, 1217, 1435, 1437, 352, 282, 15, 91, 976, 3454, 15, 454, 337, 414, 24, 219, 11, 282, 15, 20, 282, 213, 39, 154, 19, 59, 173, 254, 104, 21, 1123, 3454, 263, 424, 342, 22, 445, 247, 37, 89, 3223, 84, 91, 976, 84, 492, 53, 2239, 1234, 3, 43, 119, 94, 246] 



In [31]:
# On padde les séquences

from tensorflow.keras.preprocessing.sequence import pad_sequences

# On padde les séquence de mot
max_len = 100
X       = pad_sequences(list_tokenized_train, maxlen=max_len)

print("La phrase paddée sur une longueur {} est : \n {}". format(max_len, X[0]))
      

La phrase paddée sur une longueur 100 est : 
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  334   10  981
    2  143 1042 1556 1047 1217 1435 1437  352  282   15   91  976 3454
   15  454  337  414   24  219   11  282   15   20  282  213   39  154
   19   59  173  254  104   21 1123 3454  263  424  342   22  445  247
   37   89 3223   84   91  976   84  492   53 2239 1234    3   43  119
   94  246]


In [32]:
# On crée les ensembles d'apprentissage et de test

# On récupère les données correspondantes aux séquences de mots
y       = data['label']
typ     = data['type']

# Indexes des données pour l'entrainement et le test (tels que spécifiés dans data)
i_trn  = [i for i in range(len(typ)) if typ[i]=='train']
i_tst  = [i for i in range(len(typ)) if typ[i]=='test']

# Données d'entrée
X_trn = X[i_trn]
X_tst = X[i_tst]
# Labels de sorties
y_trn = y[i_trn]
y_tst = y[i_tst]

## 3. Déclaration du réseau

In [51]:
# On importe les librairies pour le RNN
from tensorflow.keras.layers import Dense , Input , SimpleRNN, LSTM , Embedding, Dropout , Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 128                                                   # dimension de l'embedding
RNN_size   = 64

# On définit l'architecture du réseau
model = Sequential()
model.add(Embedding(max_features, embed_size))                     # layer embedding
model.add((LSTM(RNN_size, return_sequences = True)) )
model.add((LSTM(RNN_size, return_sequences = True)) )
model.add((LSTM(RNN_size, return_sequences = False)) )
           # layer RNN
model.add(Dropout(0.5))                                            # layer Dropout
model.add(Dense(1))                                                # layer Dense

# On affiche l'architecture de notre modèle
model.summary()

# On spécifie la fonction de perte, l'optimiseur, et la fonction d'évaluation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])




Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 128)         640000    
_________________________________________________________________
lstm_20 (LSTM)               (None, None, 64)          49408     
_________________________________________________________________
lstm_21 (LSTM)               (None, None, 64)          33024     
_________________________________________________________________
lstm_22 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 755,521
Trainable params: 755,521
Non-trainable params: 0
_______________________________________________

## 4. Entrainement du réseau

In [ ]:
# On entraine le réseau
batch_size = 32                                                             # tailles des mini-batch
epochs = 5        
opt = optimizers.Adam(learning_rate=0.01)                                                           # nombre d'époques
history = model.fit(X_trn,y_trn, batch_size=batch_size, epochs=epochs, validation_split=0.2) # on entraine

Epoch 1/5
264/625 [===========>..................] - ETA: 59s - loss: 0.3936 - accuracy: 0.8589

In [36]:
# On prédit sur l'ensemble de test

# On prédit sur les données de test
y_hat = model.predict(X_tst)

# On tranforme les prédictions en labels
i_pos = [i for i in range(len(y_hat)) if y_hat[i]>0]
i_neg = [i for i in range(len(y_hat)) if y_hat[i]<=0]

y_pred   = np.zeros(len(y_hat))
y_pred[i_pos] = 1
y_pred[i_neg] = 0


In [37]:
# On importe les librairies pour l'évaluation
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

# On calcule la matrice de confusion
cm_test = confusion_matrix(y_tst, y_pred)
print('La matrice de confusion sur le jeu de test :\n', cm_test, '\n')

# On calcul le score d accuracy
acc_train=accuracy_score(y_tst, y_pred)
print('L accuracy sur le jeu de test est :\n', acc_train)


La matrice de confusion sur le jeu de test :
 [[ 4131  8369]
 [  183 12317]] 

L accuracy sur le jeu de test est :
 0.65792
